# 1. Import

In [1]:
# notebook settings flags
isDebug = False
isExperimental = False

In [ ]:
isColab = False
# TO SHOW INTERACTIVE PLOT
if 'google.colab' in str(get_ipython()):
    !pip install astroquery
    !pip install ipympl # interactive figure
    %matplotlib widget
    from google.colab import output
    output.enable_custom_widget_manager()
    isColab = True
else:
    %matplotlib qt

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance
from astropy.table import QTable, Table, Column
from astroquery.gaia import Gaia
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

Status messages could not be retrieved


# 2. Astroquery
Query the dataset(s) using the [provided API](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html). (Uses SQL queries)

It is possible to create SQL queries with the [Gaia Archive](https://gea.esac.esa.int/archive/):
1. *Search* Tab
2. fill out input
3. *Show Query*

## 2.1. Query NASA Exoplanets Archive
Get a list of exoplanets with the relevant information:
- planet name
- right ascension
- declination
- (parallax)
- distance

### 2.1.a List of all Exoplanets in the archive

In [4]:
# get planet name, right ascension, declination, parallax & distance from NASA exoplanet archive
query_exoplanets = """
SELECT TOP 100000
    ra, dec, sy_plx, sy_dist
FROM nasa_exoplanet_archive.pscomppars
WHERE sy_dist IS NOT NULL
ORDER BY sy_dist ASC
"""

results_exoplanets = NasaExoplanetArchive.query_criteria(table="pscomppars", select="pl_name, ra, dec, sy_plx, sy_dist, gaia_id", where="sy_dist is not Null", order="sy_dist")

In [5]:
def select_exoplanet(id):
    return results_exoplanets[id]

# backend selects toi-700d
exoplanet = select_exoplanet(494)

if isDebug:
    print(exoplanet)

### 2.1.b Find the corresponding star
Search the gaia database for the star in the exoplanet's system.

In [6]:
# find corresponding star
if isExperimental:
    query_exoplanet_star = f"""SELECT gaia_source.designation,gaia_source.ra,gaia_source.dec,gaia_source.parallax,gaia_source.phot_g_mean_mag,gaia_source.distance_gspphot
    FROM gaiadr3.gaia_source
    WHERE gaia_source.designation LIKE '{exoplanet['gaia_id']}'
    ORDER BY gaia_source.distance_gspphot ASC;
    """
    job_exoplanet_star = Gaia.launch_job(query_exoplanet_star)
    result_exoplanet_star = job_exoplanet_star.get_results()

    if isDebug:
        print(result_exoplanet_star)

    # fallback if gaia dr3 doesn't contain the star
    if len(result_exoplanet_star) == 0:
        query_exoplanet_star_dr2 = f"""SELECT gaia_source.designation,gaia_source.ra,gaia_source.dec,gaia_source.parallax,gaia_source.phot_g_mean_mag
        FROM gaiadr2.gaia_source
        WHERE gaia_source.designation LIKE '{exoplanet['gaia_id']}'
        ORDER BY gaia_source.parallax ASC;
        """

        job_exoplanet_star = Gaia.launch_job(query_exoplanet_star_dr2)
        result_exoplanet_star = job_exoplanet_star.get_results()

        # calculate missing distance
        if result_exoplanet_star:
            result_exoplanet_star['distance_gspphot'] = Distance(parallax=result_exoplanet_star['parallax'].filled(np.nan), allow_negative=True)

    if isDebug:
        print(result_exoplanet_star)


## 2.2. Query Gaia Star Catalogue
Get a table of stars within the specified search region with their relevant information:
- right ascension
- declination
- parallax
- distance
- magnitude

In [ ]:
target_ra = exoplanet['ra'].value
target_dec = exoplanet['dec'].value

# experimental stuff for offseting the search cone
if isExperimental:
	exoplanet_coords = SkyCoord(exoplanet['ra'], exoplanet['dec'], distance=exoplanet['sy_dist'], frame='icrs')
	star_coords = SkyCoord(result_exoplanet_star['ra'], result_exoplanet_star['dec'], distance=result_exoplanet_star['distance_gspphot'], frame='icrs')
	pa = star_coords.position_angle(exoplanet_coords).to(u.deg)
	sep = star_coords.separation(exoplanet_coords)
	offset = star_coords.directional_offset_by(pa, sep/2)
	print(pa, sep, offset)

	dra, ddec = star_coords.spherical_offsets_to(exoplanet_coords)
	print(dra, ddec)

	cone_target = exoplanet_coords.spherical_offsets_by(dra * 100, ddec * 100)
	print(cone_target)

	# shift target
	target_ra += dra * 1000
	target_dec += ddec * 1000

# Query Gaia Data for Earth
query_stars = """
SELECT TOP 1000000 gaia_source.designation,gaia_source.ra,gaia_source.dec,gaia_source.parallax,gaia_source.phot_g_mean_mag,gaia_source.distance_gspphot
FROM gaiadr3.gaia_source
WHERE parallax IS NOT NULL
AND CONTAINS(
	POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),
	CIRCLE('ICRS',""" + str(target_ra) + """,""" + str(target_dec)+ """,90)
)=1
AND  (gaiadr3.gaia_source.phot_g_mean_mag<10)
ORDER BY gaia_source.distance_gspphot ASC
"""

job_stars = Gaia.launch_job(query_stars)
results_stars = job_stars.get_results()

if isDebug:
	print(results_stars)

# define boundaries
lower_bound = 1 * u.lyr
upper_bound = 1 * u.lyr

lower_bound_plx = lower_bound.to(u.mas, equivalencies=u.parallax())
upper_bound_plx = upper_bound.to(u.mas, equivalencies=u.parallax())

# distance in parsec (1pc = 3.26ly)
distance_minimum = (exoplanet['sy_dist'] - lower_bound).value
distance_maximum = (exoplanet['sy_dist'] + upper_bound).value

# parallax as fallback as some stars don't have any distance values (1 parsec = 1 arcsecond)
parallax_minimum = (exoplanet['sy_plx'] - lower_bound_plx).value
parallax_maximum = (exoplanet['sy_plx'] + upper_bound_plx).value

# query only stars further away than exoplanet (larger distance / smaller parallax)
query_stars_filtered ="""
SELECT TOP 1000000 gaia_source.designation,gaia_source.ra,gaia_source.dec,gaia_source.parallax,gaia_source.phot_g_mean_mag,gaia_source.distance_gspphot
FROM gaiadr3.gaia_source
WHERE
CONTAINS(
	POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),
	CIRCLE('ICRS',""" + str(target_ra) + """,""" + str(target_dec)+ """,90)
)=1  AND  ((gaiadr3.gaia_source.distance_gspphot BETWEEN """+ str(distance_minimum)+""" AND """ + str(distance_maximum) + """) OR (gaiadr3.gaia_source.parallax BETWEEN""" + str(parallax_minimum)+""" AND """ + str(parallax_maximum) + """))
AND (gaiadr3.gaia_source.parallax >= 0)
ORDER BY gaia_source.distance_gspphot ASC, gaia_source.parallax ASC
"""
job_stars_filtered = Gaia.launch_job(query_stars_filtered)
results_stars_filtered = job_stars_filtered.get_results()

if isDebug:
	print(results_stars_filtered)

In [8]:
# calculate distances to exoplanet
def calculate_distances(stars_dist, stars_dec, stars_ra, point_dist, point_dec, point_ra):
    return np.sqrt(point_dist**2 + stars_dist ** 2 - 2*point_dist*stars_dist * (np.cos(point_dec) * np.cos(stars_dec) * np.cos(point_ra - stars_ra) + np.sin(point_dec) * np.sin(stars_dec)))

# astropy distance calculation from earth
distance = Distance(parallax=results_stars_filtered['parallax'].filled(np.nan), allow_negative=True)


if isDebug:
    # to earth for testing
    earth = [exoplanet['ra'], exoplanet['dec'], 0.0]
    distances_earth = calculate_distances(distance.value, results_stars_filtered['dec'], results_stars_filtered['ra'], earth[2], earth[1], earth[0])
    print(f"earth distances: {distances_earth}")

# to exoplanet
distances = calculate_distances(distance.value, results_stars_filtered['dec'], results_stars_filtered['ra'], float(exoplanet['sy_dist'].value), float(exoplanet['dec'].value), float(exoplanet['ra'].value))

if isDebug:
    print(f"exoplanet distances: {distances}")

# some stars don't have any distance data so use the value calculated from the parallax
results_stars_filtered['distance_gspphot'][np.isnan(results_stars_filtered['distance_gspphot'])] = distance[np.isnan(results_stars_filtered['distance_gspphot'])]

In [9]:
# absolute magnitude
results_stars_filtered['abs_magnitude'] = results_stars_filtered['phot_g_mean_mag'] - 5 * np.log((Distance(parallax=results_stars_filtered['parallax']) - 1 * u.pc).value)
if isDebug:
    print(results_stars_filtered['phot_g_mean_mag'])
    print(results_stars_filtered['abs_magnitude'])

In [10]:
# convert to cartesian
def galactic_to_cartesian(ra, dec, parallax, distance=None):
    if distance is None:
        distance = Distance(parallax=parallax, allow_negative=True) # some parallaxes were negative (maybe they shouldn't be?)
    coords = SkyCoord(ra=ra, dec=dec, distance=distance).represent_as('cartesian') # convert to cartesian
    return coords.x, coords.y, coords.z

def shift_coordinates(ras, decs, parallaxes, r0, theta0, phi0):
    # calculate galatic coordinates
    xs, ys, zs = galactic_to_cartesian(ras, decs, parallaxes)

    # astropy to cartesian
    coord_exo = SkyCoord(ra=phi0, dec=theta0, distance=r0, unit=(u.deg, u.deg, u.pc)).represent_as('cartesian')

    # shift coordinates
    x_new = xs - coord_exo.x
    y_new = ys - coord_exo.y
    z_new = zs - coord_exo.z

    # astropy to spherical
    coords_cartesian = SkyCoord(x_new, y_new, z_new, representation_type='cartesian')
    coords_spherical = coords_cartesian.represent_as('spherical')

    # return all
    return coords_spherical, coords_cartesian

# shift coordinates
coords, coords_cartesian = shift_coordinates(results_stars_filtered['ra'], results_stars_filtered['dec'], results_stars_filtered['parallax'], float(exoplanet['sy_dist'].value), float(exoplanet['dec'].value), float(exoplanet['ra'].value))



# spherical coordinates filtered table
star_table = Table()
star_table['ra'] = coords.lon
star_table['dec'] = coords.lat
star_table['distance_gspphot'] = coords.distance
star_table['phot_g_mean_mag'] = results_stars_filtered['abs_magnitude'] + 5 * np.log(np.abs(star_table['distance_gspphot'].value - 1)) # recalculate apparent magnitude

def calculate_distances_cartesian(x,y,z,x0,y0,z0):
    return np.sqrt(x**2 + y**2 + z**2)

# cartesian coordinates filtered table
star_table_cartesian = QTable()
star_table_cartesian['x'] = coords_cartesian.x
star_table_cartesian['y'] = coords_cartesian.y
star_table_cartesian['z'] = coords_cartesian.z
star_table_cartesian['distance_gspphot'] = calculate_distances_cartesian(coords_cartesian.x, coords_cartesian.y, coords_cartesian.z, 0,0,0)
star_table_cartesian['phot_g_mean_mag'] = results_stars_filtered['abs_magnitude'] + 5 * np.log(np.abs(star_table_cartesian['distance_gspphot'].value - 1)) # recalculate apparent magnitude

if isDebug:
    # test to compare with previously calculated distances between exoplanet & the stars
    table = QTable()
    table['dist'] = coords.distance
    table.pprint()

    # test distance of closest star to exoplanet through separation
    c1 = SkyCoord(exoplanet['ra'], exoplanet['dec'], distance=exoplanet['sy_dist'], frame='icrs')
    c2 = SkyCoord(results_stars['ra'][0], results_stars['dec'][0], distance=results_stars['distance_gspphot'][0], unit=(u.deg, u.deg, u.pc), frame='icrs')
    sep = c2.separation_3d(c1)
    print(f"separation/distance: {sep.value}")

    # distance error after conversion
    error = table['dist'] - distances * u.pc
    print(error)
    error_max = max(abs(error))
    error_max_id = np.argmax(abs(error))
    print(f"max error: {error_max}, id: {error_max_id}")
    print(results_stars_filtered[error_max_id])
    print(table[error_max_id])
    print(star_table[error_max_id])
    print(star_table_cartesian[error_max_id])


# 3. Create a Star Chart of the night sky from chosen Exoplanet
Create an interactive 2D image of what the night sky might look like by plotting the stars on a 2D plane. Represent stars with size depending on their apparent magnitude.

Stars are only visible until a certain magnitude. The apparent magnitude of a night sky object is a measure of its brightness as seen from Earth.
The scale of the magnitude is logarithmic reversely, which means that the brighter a star is, the lower the magnitude level.

Visible for:
- naked eye: ~6.5
- 50mm binoculars: ~9
- amateur telescope: ~13

src: [some random article](https://levelup.gitconnected.com/how-to-use-python-to-create-custom-star-maps-for-your-next-stargazing-journey-9908b421f30e)

In [ ]:
def create_star_chart(stars, max_star_size, x_lim_lower, x_lim_upper, draw_constellation=False):
    # filter not visible stars (from earth)
    limiting_magnitude = 9
    bright_stars = (stars['phot_g_mean_mag'] <= limiting_magnitude)
    magnitude = stars['phot_g_mean_mag'][bright_stars]

    # adjust size of marker based on magnitude
    marker_size = max_star_size * 10 ** (magnitude / -2.5)

    # plot stars
    with plt.rc_context({'axes.edgecolor':'white', 'xtick.color':'white', 'ytick.color':'white', 'figure.facecolor':'white'}): #adjust (background-)colors
        fig, ax = plt.subplots(figsize=(12, 12), facecolor='black')
        cs = ax.scatter(stars['ra'][bright_stars], stars['dec'][bright_stars], s=marker_size, color='white', marker='.', zorder=2)
        #fig.tight_layout(pad=0)
        ax = plt.gca()
        ax.set_xlim(ax.get_xlim()[::-1]) # inverse axis to match reference images of Aquila
        ax.set_aspect('equal')
        plt.title(f"Night Sky as seen from Earth", fontsize=18, color='yellow')
        ax.set_facecolor('black')
        plt.margins(x=0,y=0)
        ax.set_xlim(x_lim_lower, x_lim_upper)
        plt.axis('off')
        plt.show()

def create_star_chart_filtered(stars, max_star_size, distance, draw_constellation=True):
    # filter not visible stars (from exoplanet)
    limiting_magnitude = 6.5
    bright_stars = (stars['phot_g_mean_mag'] <= limiting_magnitude)
    magnitude = stars['phot_g_mean_mag'][bright_stars]

    # adjust size of marker based on magnitude
    marker_size = max_star_size * 10 ** (magnitude / -2.5)

    # plot stars
    with plt.rc_context({'axes.edgecolor':'white', 'xtick.color':'white', 'ytick.color':'white', 'figure.facecolor':'white'}): #adjust (background-)colors
        fig, ax = plt.subplots(figsize=(12, 12), facecolor='black')
        cs = ax.scatter(stars['ra'][bright_stars], stars['dec'][bright_stars], s=marker_size, color='white', marker='.', zorder=2)

        ax = plt.gca()
        ax.set_xlim(ax.get_xlim()[::-1]) # inverse axis to match reference images of Aquila
        ax.set_aspect('equal')
        plt.title(f"Night Sky as seen from {exoplanet['pl_name']}", fontsize=18, color='yellow')
        ax.set_facecolor('black')
        plt.axis('off')
        plt.show()


max_star_size=200 #bigger == brighter == more stars, try e.g. 200


# limit the number of stars to only those which are within the selected fov
deviation = 15
ras = results_stars['ra']
decs = results_stars['dec']
lower_ra = exoplanet['ra'].value - deviation
upper_ra = exoplanet['ra'].value + deviation
lower_dec = exoplanet['dec'].value - deviation
upper_dec = exoplanet['dec'].value + deviation
l_ra = [ras > lower_ra]
u_ra = [ras < upper_ra]
mask_ra = np.array(l_ra) & np.array(u_ra)

l_dec = [decs > lower_dec]
u_dec = [decs < upper_dec]
mask_dec = np.array(l_dec) & np.array(u_dec)
mask = mask_ra & mask_dec

result_stars_lim = results_stars[mask.flatten()]
create_star_chart(result_stars_lim, max_star_size, lower_ra, upper_ra)


# remove the brightest star
isExperimental = True
if isExperimental:
    closest_star_id = np.argmin(star_table['phot_g_mean_mag'])
    if isDebug:
        print(f"id: {closest_star_id}")
    star_table_wo_star = star_table.copy()
    star_table_wo_star['phot_g_mean_mag'][closest_star_id] += 10000000
    create_star_chart_filtered(star_table_wo_star, max_star_size / 100, exoplanet['sy_dist'], False)
else:
    create_star_chart_filtered(star_table, max_star_size / 100, exoplanet['sy_dist'], False) # reduced marker size for exoplanet perspective (a little bit cheating?)
isExperimental = False

# 4. Create A 3D plot to Verify Coordinates
Use 3D plot to look at how the stars are distributed in 3D space.

In [ ]:
def create_3D_plot(stars, max_star_size, draw_constellation=True, shifted=False):
    # filter not visible stars (from exoplanet)
    # limiting_magnitude = 6
    # bright_stars = (stars['phot_g_mean_mag'] <= limiting_magnitude)
    # magnitude = stars['phot_g_mean_mag'][bright_stars]

    # adjust size of marker based on magnitude
    marker_size = max_star_size * 10 ** (stars['phot_g_mean_mag'] / -2.5)
    #marker_size = [float(size) for size in marker_size] # it was complaining about not b

    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(projection='3d'), layout='compressed', facecolor='#041A40')
    planet = SkyCoord(exoplanet['ra'], exoplanet['dec'], distance=exoplanet['sy_dist']).represent_as('cartesian')

    # data
    xData = stars['x']
    yData = stars['y']
    zData = stars['z']
    ax.set_facecolor('black')
    ax.grid(False)
    plt.axis('off')

    ax.scatter3D(xData, yData, zData, s=marker_size, color='white')
    if not shifted:
        ax.scatter3D(0,0,0, s=20, color='blue')
        ax.scatter3D(planet.x, planet.y, planet.z, s=50, color='red')
    else:
        ax.scatter3D(0,0,0, s=50, color='red')

# don't run in colab too laggy to move 3D plot around anyways
if not isColab:
    # create cartesian table for earth perspective
    earth_coords_cartesian = SkyCoord(results_stars_filtered['ra'], results_stars_filtered['dec'], results_stars_filtered['distance_gspphot']).represent_as('cartesian')

    star_table_earth_cartesian = QTable()
    star_table_earth_cartesian['x'] = earth_coords_cartesian.x
    star_table_earth_cartesian['y'] = earth_coords_cartesian.y
    star_table_earth_cartesian['z'] = earth_coords_cartesian.z
    star_table_earth_cartesian['phot_g_mean_mag'] = results_stars_filtered['phot_g_mean_mag'].value
    star_table_earth_cartesian['distance_gspphot'] = results_stars_filtered['distance_gspphot']

    # adjust size of stars
    max_star_size=1

    create_3D_plot(star_table_cartesian, max_star_size / 100, False, True)
    create_3D_plot(star_table_earth_cartesian, max_star_size * 100, False, False)